In [ ]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(Path(os.getcwd()).parent))

In [ ]:
from loader import Power, Weather
from constant import FeatureType
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.python.keras.optimizer_v2.rmsprop import RMSProp
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error,r2_score
from datetime import datetime, timedelta
import hpbandster.core nameserver as hpns
import hpbandster.core.result as hpres
from hpbandster.optimizers import BOHB as BOHB

import json
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Path ====== #
args.root = root

# ====== Model ====== #
args.frame_in = 72
args.frame_out = 24
args.nameserver = '127.0.0.1'
args.min_budget = 32
args.max_Budget = 512
args.n_iterations = 20

# ====== Data ====== #
args.years = [2017, 2018, 2019]
args.region = "Jindo"
args.station = 192
args.ratio = [0.6, 0.2, 0.2]

# ====== Features ====== #
features = [FeatureType.SUNSHINE,
            FeatureType.GROUND_TEMPERATURE,
            FeatureType.HUMIDITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.PRECIPITATION,
            FeatureType.STEAM_PRESSURE,
            FeatureType.DEW_POINT_TEMPERATURE,
            FeatureType.ATMOSPHERIC_PRESSURE]

In [ ]:
power = Power(args)
power_data = power.get_data()

weather = Weather(args, features)

for i in range(len(features)):
    weather_data = weather.get_data(i+1)
    setattr(args, 'feature_len', i + 1)
    setattr(args, 'name', 'automl_feature_set_%d' % (i + 1))

    train = [weather_data['train'], power_data['train']]
    val = [weather_data['val'], power_data['val']]
    test = [weather_data['test'], power_data['test']]

    dataset = {'train': train, 'val': val, 'test': test}

    name_server = hpns.NameServer(run_id=args.name, host=args.nameserver, port=None)
    name_server.start()
    
    worker = SolarWorker(sleep_interval=0, namewerver=args.nameserver, run_id=args.name)
    worker.set_dataset(args, dataset)
    worker.run(background=True)
    
    bohb = BOHB(configspace = worker.get_configspace(),
               run_id=args.name, nameserver=args.nameserver,
               min_budget=args.min_budget, max_budget=args.max_budget)
    
    result = bohb.run(n_iterations=args.n_iterations)
    
    bohb.shutdown(shutdown_workers=True)
    name_server.shutdown()
    
    id2config = result.get_id2config_mapping()
    incumbent = res.get_incumbent_id()
    
    print('Best found configurations:', id2config[incumbent]['config'])
    print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
    print('A total of %i runs where excuted.' % len(result.get_all_runs()))
    print('Total budget corresponds to %.1f full function evaluations.' % (sum([r.budget for r in result.get_all_runs()])/args.max_budget))